In [8]:
def genOrgan(gene,organ):
    
    import requests
    import json
    import pandas as pd

    query = {}

    # queries must be formatted in Lucene ElasticSearch style
    # https://www.elastic.co/guide/en/elasticsearch/reference/current/full-text-queries.html

    # here are different query types :

    # MULTI_MATCH queries
    # use it for searching in specific fields, with a general OR or AND

    search_fields = ["title","abstract","keywords","mesh_terms"]
    # you can use "title^3" for boosting by 3 scores in title
    keywords = gene + " " + organ
    my_operator = "and" # default is "or"
    my_type = "phrase" # use it for phrase matching (exact expression)

    my_query = {
        "size": 10, # maximum amount of hits returned
        "from" : 0, # offset from the first result you want to fetch
        "query": {
            "multi_match" : {
                "query" : keywords,
                "fields" : search_fields
                #,"operator" : my_operator
                #,"type" = "phrase"
                }
            }
    }
    
    # query for required gene and organ
    
    my_query = {
    "query": {
        "bool" : {
            "must" :[ # AND
                {
                    "bool" : {
                        "should" :[ # OR
                            {
                                "match" : {"title": gene},
                                "match" : {"title": organ}
                                }
                            ]
                        }
                    },
                {
                    "bool" : {
                        "should" :[ # OR
                            {
                                "match" : {"abstract": gene},
                                "match" : {"abstract": organ}
                                }
                            ]
                        }
                    },
                ]
            }
        }
    }
    
    # call
    url_API = "http://candy.hesge.ch/SIBiLS/MEDLINE/search.jsp"
    my_json_query = json.dumps(my_query) # json to string
    my_params = {"json_query": my_json_query} # parameters dictionary
    r = requests.post(url = url_API, params = my_params)

    # get response and print in output file
    response = r.text
    with open("SIBiLS_MED_search.json","w",encoding="utf-8") as file:
       file.write(r.text)

    parsed = json.loads(response)
    
    data = []
    for list_item in parsed['hits']:
        source_item = list_item['_source']
        temp_list = [source_item['pmid'], source_item['title'], source_item['abstract']]
        data.append(temp_list)
    
    df = pd.DataFrame(data, columns = ['PMID', 'Title', 'Abstract'])
    return df

In [12]:
df1 = genOrgan('alk','lungs')
df1

,PMID,Title,Abstract
0,6844355,Glycosaminoglycan composition of germfree dog ...,Connective tissue of the lung is fundamentally...
1,28719637,Microbiome overview in swine lungs.,Mycoplasma hyopneumoniae is the etiologic agen...
2,4060481,[Complicated abscesses of the lungs].,The work is based upon the treatment of 445 pa...
3,6520031,Regional interdependence in calf lungs.,"We measured K, an index of interdependence in ..."
4,2675409,Expression of MHC antigens in normal human lun...,Immunocytochemical techniques have been used t...
5,645607,Isolated perfused dog lungs: a comparative ste...,Established stereologic techniques were used t...
6,20177723,Advances in artificial lungs.,Artificial lungs have already been developed a...
7,6175261,Glycosaminoglycans in emphysematous and fibrot...,Amounts and distributions of glycosaminoglycan...
8,7075785,Diffusion in avian lungs.,"Arterial PO2 in resting, normoxic avian lungs ..."
9,946912,Alveolar stability and phospolipid content in ...,In pig lungs with enzootic pneumonia (a mycopl...
